In [1]:
import os 
stories = ''
data_dir = r'F:\works\A-important\A-neurals\Vortex-Language-Models\GPT-from_SCRATCH\Data'
for story in os.listdir(data_dir):
    with open(os.path.join(data_dir, story), 'r', encoding='utf-8') as file:
        stories += file.read()
print(stories[:256])


At the time when Brahmadatt
was the ruler of Banaras, the
Bodhisatva was employed as the
king's procurator.

The procurator's job was to
buy the various properties and
commodities that were required
for state purposes. He had to be
an expert in estimating


In [2]:
len(stories)

97058

In [3]:
vocab = sorted(set(stories))
itos = {i: j for i, j in enumerate(vocab)}
stoi = {j: i for i, j in enumerate(vocab)}

In [4]:
def encoder(s):
    res = []
    for i in s:
        res.append(stoi[i])
    return res

def decoder(s):
    res = []
    for i in s:
        res.append(itos[i])
    return ''.join(res)

In [5]:
encoder('omg')
decoder(encoder('omg'))

'omg'

In [6]:
import torch 

data = torch.tensor(encoder(stories))
print(data.shape, data.dtype, data[:100], decoder(data[:100].tolist()))

torch.Size([97058]) torch.int64 tensor([ 0, 27, 73,  1, 73, 61, 58,  1, 73, 62, 66, 58,  1, 76, 61, 58, 67,  1,
        28, 71, 54, 61, 66, 54, 57, 54, 73, 73,  0, 76, 54, 72,  1, 73, 61, 58,
         1, 71, 74, 65, 58, 71,  1, 68, 59,  1, 28, 54, 67, 54, 71, 54, 72, 11,
         1, 73, 61, 58,  0, 28, 68, 57, 61, 62, 72, 54, 73, 75, 54,  1, 76, 54,
        72,  1, 58, 66, 69, 65, 68, 78, 58, 57,  1, 54, 72,  1, 73, 61, 58,  0,
        64, 62, 67, 60,  6, 72,  1, 69, 71, 68]) 
At the time when Brahmadatt
was the ruler of Banaras, the
Bodhisatva was employed as the
king's pro


In [7]:
train_size = int(len(data) * 0.9)
train_size

87352

In [8]:
train_data = data[:train_size]
val_data = data[train_size:] 

In [9]:
batch_size = 4 
context_lenght = 8
torch.manual_seed(12313)
def get_batch(split,):
    split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_lenght, (4, ))
    x = torch.stack([data[i : i+context_lenght] for i in ix])
    y = torch.stack([data[i + 1 : 1+i+context_lenght] for i in ix])
    return x, y

In [10]:
xb, yb = get_batch('val')
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [11]:
for batch in range(batch_size):
    for context in range(context_lenght):
        content = xb[batch, :context+1]
        target = yb[batch, context]
        print(f'when content is {content.tolist()}\tthe result will be {target.tolist()}')

when content is [11]	the result will be 1
when content is [11, 1]	the result will be 90
when content is [11, 1, 90]	the result will be 1
when content is [11, 1, 90, 1]	the result will be 39
when content is [11, 1, 90, 1, 39]	the result will be 68
when content is [11, 1, 90, 1, 39, 68]	the result will be 73
when content is [11, 1, 90, 1, 39, 68, 73]	the result will be 61
when content is [11, 1, 90, 1, 39, 68, 73, 61]	the result will be 0
when content is [74]	the result will be 71
when content is [74, 71]	the result will be 0
when content is [74, 71, 0]	the result will be 64
when content is [74, 71, 0, 64]	the result will be 62
when content is [74, 71, 0, 64, 62]	the result will be 73
when content is [74, 71, 0, 64, 62, 73]	the result will be 61
when content is [74, 71, 0, 64, 62, 73, 61]	the result will be 1
when content is [74, 71, 0, 64, 62, 73, 61, 1]	the result will be 54
when content is [57]	the result will be 71
when content is [57, 71]	the result will be 58
when content is [57, 7

In [12]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 

In [13]:
class BigramModel(nn.Module):  
    def __init__(self, vocab): 
        super(BigramModel, self).__init__() 
        self.embeddings = nn.Embedding(vocab, vocab)
    
    def forward(self, idx, target=None): 
        logits = self.embeddings(idx) 
        B, T, C = logits.shape
        if target is None: 
            loss = None
        else:
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)
        return logits, loss
    
    def Generate(self, max_len, idx):
        for _ in range(max_len):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            prob = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(prob, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
model = BigramModel(len(vocab))
logits, loss = model(xb, yb)
loss

tensor(4.8875, grad_fn=<NllLossBackward0>)

In [14]:
decoder(model.Generate(idx = torch.zeros((1, 1), dtype=torch.long), max_len=100)[0].tolist())

"\nF(p mqFHJ”™,é©w=nt3k{mPa|*dE ,I={p)MPiC«eZY;©Rs ')+jJ[)MO©_Qk»4N‘?O;”a»<:3Oy-r?!QSV«Oa&)jMr<k“4shMA0"

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [16]:
sum(p.numel() for p in model.parameters())

8649

In [17]:
for steps in range(1000):  
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


3.9882357120513916


In [18]:
print(decoder(model.Generate(idx = torch.zeros((1, 1), dtype=torch.long), max_len=100)[0].tolist()))


"NYHb5nb,#2b)—hz”uKjJ3)—sc,H™érc
TR—lonlOo;n©EzDOffB.YB©R~~v tV:GDw={Hb2©Clu»o&b#u«UGrPFH!nId
G*LTcy


In [19]:
import torch

# Set batch size, time steps, and channels
b, t, c = 2, 3, 4

# Create random tensors
tensor1 = torch.randn(b, t, c)
tensor2 = torch.randn(b, c, t)

# Perform batched matrix multiplication
result = tensor1 @ tensor2

print("Tensor 1 shape:", tensor1.shape)
print("Tensor 2 shape:", tensor2.shape)
print("Result shape:", result.shape)

Tensor 1 shape: torch.Size([2, 3, 4])
Tensor 2 shape: torch.Size([2, 4, 3])
Result shape: torch.Size([2, 3, 3])


In [20]:
tensor1.shape

torch.Size([2, 3, 4])

In [21]:
tensor1.transpose(-1, -2).shape

torch.Size([2, 4, 3])

In [22]:
tensor1.transpose(-2, -1).shape

torch.Size([2, 4, 3])

In [23]:
tensor1.transpose(-2, -1).shape

torch.Size([2, 4, 3])

In [25]:
len(vocab)

93

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [43]:
vocab_size = 93
embedding_dimension = 100

batch_size = 16
block_size = 32

num_head = 4

N = 4

In [118]:
class HEAD(nn.Module):
    def __init__(self, head_size):
        super().__init__()        
        self.key = nn.Linear(embedding_dimension, head_size, bias=False)
        self.query = nn.Linear(embedding_dimension, head_size, bias=False)
        self.value = nn.Linear(embedding_dimension, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    def forward(self, x):    
        B,T,C = x.shape
        k = self.key(x)   
        q = self.query(x)  
        wei = q @ k.transpose(-2,-1) * C**-0.5 # 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1)  
        v = self.value(x)  
        out = wei @ v  
        return out


In [119]:
class MULTIHEAD(nn.Module):
    def __init__(self, num_head, head_size):
        super().__init__()
        self.head = nn.ModuleList([HEAD(head_size) for i in range(num_head)]) 
        self.linear = nn.Linear(embedding_dimension, embedding_dimension)
        
    def forward(self, x):
        self.x = torch.cat([H(x) for H in self.head], dim= -1)
        self.x = self.linear(self.x)
        return self.x
    
    
            
        

In [120]:
class FEEDFORWARD(nn.Module):
    def __init__(self, embedding_dimension):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(embedding_dimension, embedding_dimension*10), 
            nn.ReLU(),
            nn.Linear(10*embedding_dimension, embedding_dimension)
        )
        
    def forward(self, x):
        return self.layer(x)

In [121]:
class BLOCKS(nn.Module):
    def __init__(self, embedding_dimension, num_head):
        super().__init__()
        self.head_size = embedding_dimension // num_head
        self.attention_module = MULTIHEAD(num_head, self.head_size)
        self.feed_forward_module = FEEDFORWARD(embedding_dimension)
        self.layer_norm = nn.LayerNorm(embedding_dimension)
    def forward(self, x):
        # print("-----------------------")
        # print(x)
        x = x + self.attention_module(self.layer_norm(x))
        x = x + self.feed_forward_module(self.layer_norm(x))
        return x

In [122]:

class shell_of_GPT(nn.Module):
    def __init__(self, vocab_size = vocab_size, embedding_dimension = embedding_dimension):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embedding_dimension)
        self.positional_embedding = nn.Embedding(block_size, embedding_dimension)
        self.BLOCK = nn.Sequential(*[BLOCKS(embedding_dimension, num_head) for _ in range(N)])
        self.layer_norm = nn.LayerNorm(embedding_dimension)
        self.final_linear = nn.Linear(embedding_dimension, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding(idx) 
        pos_emb = self.positional_embedding(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb
        # print("-------------------------------------")
        # print(x)
        x = self.BLOCK(x) 
        x = self.layer_norm(x)
        logits = self.final_linear(x) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) 
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [123]:
model = shell_of_GPT().to(device)

In [124]:
model

shell_of_GPT(
  (token_embedding): Embedding(93, 100)
  (positional_embedding): Embedding(32, 100)
  (BLOCK): Sequential(
    (0): BLOCKS(
      (attention_module): MULTIHEAD(
        (head): ModuleList(
          (0-3): 4 x HEAD(
            (key): Linear(in_features=100, out_features=25, bias=False)
            (query): Linear(in_features=100, out_features=25, bias=False)
            (value): Linear(in_features=100, out_features=25, bias=False)
          )
        )
        (linear): Linear(in_features=100, out_features=100, bias=True)
      )
      (feed_forward_module): FEEDFORWARD(
        (layer): Sequential(
          (0): Linear(in_features=100, out_features=1000, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1000, out_features=100, bias=True)
        )
      )
      (layer_norm): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    )
    (1): BLOCKS(
      (attention_module): MULTIHEAD(
        (head): ModuleList(
          (0-3): 4 x HEAD(
          

In [125]:
print(f"Token embedding is on: {model.token_embedding.weight.device}")
print(f"Positional embedding is on: {model.positional_embedding.weight.device}")

Token embedding is on: cuda:0
Positional embedding is on: cuda:0


In [126]:
sum([p.numel() for p in model.parameters()])

987693

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
batch_size = 4 
context_lenght = 8
torch.manual_seed(12313)
def get_batch(split,):
    split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_lenght, (4, ))
    x = torch.stack([data[i : i+context_lenght] for i in ix])
    y = torch.stack([data[i + 1 : 1+i+context_lenght] for i in ix])
    return x.to(device), y.to(device)

In [138]:
learning_rate = 1e-3
max_iters = 1000
eval_interval = 100
eval_iters = 200

torch.manual_seed(100)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    
    if iter % eval_interval == 0:
        print(f"The loss at iteration {iter} is: {loss.item():.4f}")
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

The loss at iteration 0 is: 2.7070
The loss at iteration 100 is: 1.7681
The loss at iteration 200 is: 1.5485
The loss at iteration 300 is: 2.3130
The loss at iteration 400 is: 1.7169
The loss at iteration 500 is: 1.5802
The loss at iteration 600 is: 1.7588
The loss at iteration 700 is: 1.6510
The loss at iteration 800 is: 1.7433
The loss at iteration 900 is: 1.6863


In [144]:

context = torch.ones((1, 1), dtype=torch.long, device=device)
print(decoder(model.generate(context, max_new_tokens=2000)[0].tolist()))

 A
Greatce(herede w winas t
ad ay aret agly cofrth an anghinded

tinchinth singen hined ofmen m’sty in
ch ply e,”waredinsefut aby thineyaded ongher?” bly aly winghingh, chined wwaned ikingh y an t
taly h ly dredengqonemesengheredededeng find ptoron was M
m onghity bly y’ thiexchareadech, achin thane

nde tof seved ouravede t w thizis on thendy fe hisal
she ouly’ here y, mendiksthatikin In he. w
hed roref arene
dedearexde lexthi w heve
chedinginghinchinghe hithuly out aind sty asedeled. d ofre my, ply anxtshin thinedenged ty theff anedese m
ared ireneanchmen  shunofreresarted waly orede wa
h s thale thin?"mengaly thichis n an
Hen. wamingingely Prengh bomerind sachesay pougedesurin
On futy fffalatorech Eve thederis hazely y moredeaiVin sededy ffinared, the meftly any d
ande the fouthinind alenchenese ty athine ty y en pply wad,

ting knjedenireffushiffurishindd,
. h fut arend, h,"n s cherere sarin. nedy ave Sorof y, h w h, “1 ke orin
tineshen,” 
shine w thisereron tin pederede th thingha

In [145]:
torch.save(model, 'F:\works\A-important\A-neurals\Vortex-Language-Models\GPT-from_SCRATCH\model.pth')

### Andrej's code

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
